# Functions 

## Learning Objectives 
- Define and call functions in Julia 
- Understand the use of arguments and return values in Julia functions 
- Use anonymous (lambda) functions for short snippets of functionality 
- Implement multiple dispatch by defining functions that handle different types of inputs
0 Organise code into functions for clarity and reuse, and recognise the benefits of multiple dispatches for flexibility and performance

## Defining Functions 
In Julia, you define a function using the `function` keyword (and terminate with `end`) or using a shorter one-line syntax. Functions are objects in Julia, and you can assign them to variables, pass them as arguments, etc.

A simple function definition would be: 
```Julia 
function say_hello(name)
    println("Hello, ", name, "!")
end
```

This function, `say_hello`, takes one argument `name` and prints a greeting. You would call it like `say_hello("Alice")`.

Julia can return values from functions. If you use the `return` keyword, you can return a value explicitly. If you omit `return`, Julia will return the **last expression** in the function by default. For example:  

In [1]:
# Function to square a number (explicit return)
function square(x)
    return x * x
end

# Function to cube a number (implicit return of last expression)
function cube(x)
    x
    x * x
    x * x * x   # last expression's value will be returned
end
println(square(3))
println(cube(3))

9
27


Julia also provides a concise **single-line function definition** syntax: 
```Julia 
add(a, b) = a + b
```

This defines a function `add` that returns `a+b`. It's equivalent to writing a multi-line function that returns `a+b`. You can use whichever style you find clearer; the multi-line form is helpful for longer-function bodies. 

For example, the function below is a function that can be used to calculate the area of a circle given its radius: 

In [2]:
function calculate_circle_area(radius)
    area = π * radius^2   # Julia has π predefined (or use 3.14159 for Pi)
    return area
end

r = 5.0
println("Radius: ", r, ", Area: ", calculate_circle_area(r))

Radius: 5.0, Area: 78.53981633974483


There are a number of different ways to get π within Julia, depending on the OS you are using: 
- **Julia’s built-in Unicode shortcut**: In the REPL or in most editors with the Julia extension (e.g. VS Code) you can type a backslash, `pi`, then hit Tab. Julia will replace it with the single-character π.
- **macOS**: Option + p → π
- **Windows (numeric keypad)**: Hold Alt and type 0960 on the numpad → π
- **Linux**: Ctrl + Shift + U, release, then type 03C0 and Enter → π

If you run this, you should see something like "Radius: 5.0, Area: 78.5398: We used the built-in constant `π` (pi) here for higher precision; alternatively `, 3.14` could be used. The different components of a function within Julia are: 
- **Function Name**: e.g. `calculate_circle_area` in the example above. By convention, use lowercase and underscores for multi-word names. 
- **Parameters (Arguments)**: e.g. `radius`. You can optionally add a type annotation to a parameter (like `radius::Float64`) to indicate this function is for a specific type. Still, if you leave it untyped, it will accept any types that support the operations used inside. 
- **Function Body**: the code inside, which can use the parameters. In our case, we compute `area`. 
- **Return value**: you can explicitly `return` something. If you do not, the last expression will be returned. In the circle example, we used `return` for clarity, but we could also write `π * radius^2` as the last line and omit `return`. 
After defining a function, call it by writing its name followed by arguments in parentheses, e.g. `calculate_circle_area(10)`. 

## Exercise: Writing a function 
Write a function `f_to_c(fahrenheit)` that converts a temperature from Fahrenheit to Celsius. The formula is `C = (F - 32) * 5/9`. Test your function with a couple of values; for `32`, it should return 0, and for `212`, it should return `100`.


### Keyword Arguments

Julia functions can accept **keyword arguments**, which are specified by name and given default values. These are defined after a semicolon (`;`) in the function signature:

In [3]:
function describe(x; unit="kg", precision=2)
    formatted = round(x; digits=precision)
    println("Value: $formatted $unit")
end

# Calls:
describe(12.3456)                      # uses defaults: "Value: 12.35 kg"
describe(12.3456; precision=3)         # "Value: 12.346 kg"
describe(12.3456; unit="meters")       # "Value: 12.35 meters"
describe(12.3456; unit="m", precision=1)  # "Value: 12.3 m"

Value: 12.35 kg
Value: 12.346 kg
Value: 12.35 meters
Value: 12.3 m


**Declaration**: List keyword arguments (with optional type annotations) after a semicolon: 
```julia 
function fn(pos1, pos2; kw1::T1=default1, kw2::T2=default2)
    …
end
```

**Calling Syntax**
```julia 
fn(arg1, arg2; kw2=value2, kw1=value1)
```

## Anonymous Functions

Sometimes, you need a small throwaway function to pass as an argument to another function. Julia allows creating **anonymous functions** using the -> syntax. These are similar to the "lambda" function in Python.

For example, suppose we have a list of numbers, and we want to square each of them. We could write a one-liner anonymous function for squaring: `x -> x * x`. Here, `x` is the input, and `x * x` is the output. We can pass this to a higher-order function like `map`, which applies a function to each element of a collection: 

In [4]:
numbers = [1, 2, 3, 4, 5]
squared_numbers = map(x -> x * x, numbers)
println(squared_numbers)   

[1, 4, 9, 16, 25]


In this example, `map(f, collection)` returns a new collection where function `f` has been applied to every element. We provided `map` with an anonymous function `x -> x * x`. The result `squared_numbers` is a new array of squares. 

Anonymous functions are handy for short operations, so you don't necessarily need to give a name. If the transformation is straightforward, you could also use broadcast or comprehension, but `map` with a lambda is clear and functional-style. 

You can also assign an anonymous function to a variable if you want to reuse it: 


In [5]:
squarer = x -> x^2
println(squarer(10))

100


But if you're going to name it, a standard function definition might be clearer. So typically, we use the `->` syntax inline when passing to other functions or for short-lived usage. 

**Note on anonymous functions**: Unlike named functions, anonymous functions cannot have multiple methods added to them; each anonymous function carries exactly one method. If you need multiple dispatch (i.e., different behaviours for different argument types), define a standard function with `function … end` and type‐annotated methods instead.

### Exercise: Anonymous Functions 
Using an anonymous function, create an array `evens` that contains only the even numbers from an existing array `vals = [1,2,34,8,11,14]`. Hint: You might want to use the filter function!

## Mutating Functions & the `!` Naming Convention


In Julia, any function that **mutates** (i.e. changes) one or more of its arguments should have a name ending in `!`. This "bang" signals to users that the function will perform in-place modification rather than returning a new object.

**Why use `!`?**: It makes code more readable and safer by clearly distinguishing destructive operations from pure ones.

**Common examples:**
```julia
arr = [3, 1, 2]

sort!(arr)           # mutates `arr` to become [1, 2, 3]
push!(arr, 4)        # appends 4 to `arr`
pop!(arr)            # removes and returns the last element
append!(arr, [5,6])  # concatenates another collection into `arr`
```


## Multiple Dispatch 

Multiple dispatches were introduced conceptually earlier; now, let's see how to implement them with functions. In Julia, you can define **multiple methods** for the same function name with different type signatures. Julia will dispatch (choose) the method that best matches the types of the actual arguments you pass. 

For example, imagine we want an add function that behaves differently based on argument types: 

In [6]:
# Define add for two Ints
function add(x::Int, y::Int)
    x + y
end

# Define add for two Strings (concatenate with a space in between)
function add(x::String, y::String)
    return string(x, " ", y)
end

println( add(10, 20) )        
println( add("Hello", "world!") )  


30
Hello world!


We defined two methods for `add`: one that accepts two `Int` and one that accepts two `String`. When we call `add(10, 20)`, Julia sees both arguments as `Int` and uses the integer addition method. When we call `add("Hello", "world!")`, both are `String`, so it uses the string concatenation method. The same function name, `add,` is used, but the behaviour differs by type. 

We could add more methods if needed (for example, adding an `add(x::Float64, y::Float64)` or a mixed `add(x::Int, y::Float64)` etc.) In fact, Julia's standard library often provides a rich set of methods for functions to handle different types. 

The benefits of this are: 
- **Flexibility**: You generically write code but provide special cases for specific types when needed. 
- **Performance**: Julia will pick the most specific method and compile optimised machine code for that method. 
- **Extensibility**: Users can add new methods to functions (even those from the standard library or other packages) for new types they define without modifying the original code, a form of polymorphism.

It's important to remember that there is **no dispatch on keywords**. Julia's multiple-dispatch mechanism examines **only** the type of **positional** arguments. 

#### Inspecting Methods & Handling Missing Methods

**Listing methods**: You can see all methods defined for a function with:

In [7]:
methods(all)

# 10 methods for generic function "all" from Base:
  [1] all(x::Tuple{})
     @ tuple.jl:662
  [2] all(x::Tuple{Bool})
     @ tuple.jl:663
  [3] all(x::Tuple{Bool, Bool})
     @ tuple.jl:664
  [4] all(x::Tuple{Bool, Bool, Bool})
     @ tuple.jl:665
  [5] all(B::BitArray)
     @ bitarray.jl:1722
  [6] all(a::AbstractArray; dims)
     @ reducedim.jl:994
  [7] all(f::Function, a::AbstractArray; dims)
     @ reducedim.jl:995
  [8] all(itr)
     @ reduce.jl:1193
  [9] all(f, itr::Tuple)
     @ reduce.jl:1323
 [10] all(f, itr)
     @ reduce.jl:1301

**MethodError**: If you call a function on argument types for which no method exists, Julia throws a `MethodError`. For example: 

```julia 
add(1.0, 2.0)
```

would produce the error: 
```julia
MethodError: no method matching add(::Float64, ::Float64)
The function `add` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  add(::String, ::String)
   @ Main In[6]:7
  add(::Int64, ::Int64)
   @ Main In[6]:2


Stacktrace:
 [1] top-level scope
   @ In[9]:1
```

signalling that you either need to define a new method, `add(x::Float64, y::Float64) = x + y` or convert your input to types that existing methods accept. 

### Exercise: Multiple Dispatch 
Define a function `myabs` that returns the absolute value of its argument, but implement it using multiple dispatch: one method for `Int` and one method for `String` that returns the length of the string, so that `myabs("hello")` would return `5`).

### Why Multiple Dispatch Matters 
In Julia, **every function is by default generic**; you can add methods to it. This is different from single-dispatch object-oriented languages (where typically a method belongs to one class and dispatch is only on the object instance type). Multiple dispatch is particularly powerful in mathematical code, where operations might naturally be defined by any combinations of operand types (e.g., mixing units, numeric types, etc.)

To summarise, multiple dispatch allows us to **write clearer code** (we don't need long chains of type-checking `if-else` inside a single function). The Julia compiler **ensures it runs fast** by picking the precise method and compiling it. 

## Organising Code with Functions 
It's good practice to wrap logic inside functions rather than writing everything in global code. Functions: 
- make code reusable (you can call the same code with different inputs easily)
- clarify intent (a function name can describe what the code does) 
- In Julia, functions are essential for performance (code inside functions is optimised and JIT-compiled, whereas code in global scope is more complex for the compiler to optimise. 
As you build larger programs, you'll likely have many small functions, each handling a specific task, which together solve your problem.

# End of Section Quiz

In [6]:
using JSON

function show_quiz_from_json(path)
    quiz_data = JSON.parsefile(path)

    html = """
    <style>
    .quiz-question {
        background-color: #6c63ff;
        color: white;
        padding: 12px;
        border-radius: 10px;
        font-weight: bold;
        font-size: 1.2em;
        margin-bottom: 10px;
    }

    .quiz-form {
        margin-bottom: 20px;
    }

    .quiz-answer {
        display: block;
        background-color: #f2f2f2;
        border: none;
        border-radius: 10px;
        padding: 10px;
        margin: 5px 0;
        font-size: 1em;
        cursor: pointer;
        text-align: left;
        transition: background-color 0.3s;
        width: 100%;
    }

    .quiz-answer:hover {
        background-color: #e0e0e0;
    }

    .correct {
        background-color: #4CAF50 !important;
        color: white !important;
        border: none;
    }

    .incorrect {
        background-color: #D32F2F !important;
        color: white !important;
        border: none;
    }

    .feedback {
        margin-top: 10px;
        font-weight: bold;
        font-size: 1em;
    }
    </style>

    <script>
    function handleAnswer(qid, aid, feedback, isCorrect) {
        // Reset all buttons for the question
        let buttons = document.querySelectorAll(".answer-" + qid);
        buttons.forEach(btn => {
            btn.classList.remove('correct', 'incorrect');
        });

        // Apply correct/incorrect to selected
        let selected = document.getElementById(aid);
        selected.classList.add(isCorrect ? 'correct' : 'incorrect');

        // Show feedback below the question
        let feedbackBox = document.getElementById('feedback_' + qid);
        feedbackBox.innerHTML = feedback;
        feedbackBox.style.color = isCorrect ? 'green' : 'red';
    }
    </script>
    """

    for (i, question) in enumerate(quiz_data)
        qid = "$i"
        html *= """<div class="quiz-question">$(question["question"])</div><form class="quiz-form">"""

        for (j, answer) in enumerate(question["answers"])
            aid = "q$(i)_a$(j)"
            feedback = answer["feedback"]
            correct = startswith(lowercase(feedback), "correct")
            html *= """
            <button type="button" class="quiz-answer answer-$qid" id="$aid"
                onclick="handleAnswer('$qid', '$aid', '$feedback', $(correct))">
                $(answer["answer"])
            </button>
            """
        end

        html *= """<div class="feedback" id="feedback_$qid"></div></form><hr>"""
    end

    display("text/html", html)
end


# Use the function
show_quiz_from_json("questions/summary_functions.json")

What happens if you omit the return keyword in a Julia function? 
 The function returns nothing
 
 
 The function returns the first expression inside the body
 
 
 The function raises an error unless return is specified
 
 
 The function returns the value of the last expression
 
 Which of the following is a correct way to define an anonymous function in Julia that adds 1 to its input? 
 function(x) = x + 1
 
 
 anon x -> x + 1
 
 
 x -> x + 1
 
 
 anonymous(x) = x + 1
 
 Which of the following best describes multiple dispatch in Julia? 
 A function can only accept one argument at a time
 
 
 A method is selected based only on the first argument's type
 
 
 Julia selects the method to run based on the types of all arguments
 
 
 It refers to calling multiple functions at once